In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import configparser
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = "08009030" # 店舗ID
SPREADSHEET_KEY = '1oEspZKFeuIqrdivdDBiI2HXvIshXP3NM_4tsDw2fThQ' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 481 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com "
user_pw = "paruparu22"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Users\\user\\AppData\\Local\\Google\\Chrome\\Application\\chrome.exe'
config_ini = configparser.ConfigParser()
config_ini.read('config.ini', encoding='utf-8')

wd = config_ini['ChromeDriver']['WD']

path = "C:\\Users\\user\\WD\\" + str(wd) + "\\chromedriver.exe"
chrome_service = ChromeService(executable_path=path)

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
p2row = worksheet.cell(11,  25).value
s20row = worksheet.cell(11,  28).value
s11row = worksheet.cell(11,  29).value
s2row = worksheet.cell(11,  30).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
p2table = []
s20table = []
s11table = []
s2table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                if unit == "20":
                    p2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    if unit == "20":
                        p2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2127":
                    s20table.append(li)
                if unit == "1111":
                    s11table.append(li)
                if unit == "222":
                    s2table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2127":
                        s20table.append(li)
                    if unit == "1111":
                        s11table.append(li)
                    if unit == "222":
                        s2table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'P0.2RAW'!F" + str(int(p2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p2table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S11.1RAW'!F" + str(int(s11row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s11table})
wb.values_append("'S2.2RAW'!F" + str(int(s2row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s2table})


['08/18', '025864', '1', '47436.0', '-25277.0', '15', '8']
['08/18', '025852', '2', '45641.0', '-4513.0', '48', '9']
['08/18', '025557', '3', '48718.0', '-29791.0', '8', '4']
['08/18', '025797', '4', '48462.0', '17756.0', '99', '16']
['08/18', '025779', '5', '46923.0', '10533.0', '52', '11']
['08/18', '025838', '6', '49231.0', '8126.0', '37', '12']
['08/18', '025825', '7', '36667.0', '28890.0', '40', '10']
['08/18', '025798', '8', '45897.0', '-5717.0', '28', '11']
['08/18', '025626', '9', '48462.0', '-4513.0', '62', '20']
['08/18', '025439', '10', '43077.0', '3311.0', '34', '9']
['08/18', '025430', '11', '52821.0', '-2406.0', '32', '10']
['08/18', '025806', '12', '49487.0', '-16249.0', '14', '4']
['08/18', '025801', '13', '51538.0', '-902.0', '30', '9']
['08/18', '025364', '14', '58462.0', '-8726.0', '53', '24']
['08/18', '025364', '15', '45641.0', '-7221.0', '42', '16']
['08/18', '025436', '16', '53846.0', '-8726.0', '35', '12']
['08/18', '025436', '17', '47949.0', '15348.0', '51', '1

['08/18', '025509', '140', '37949.0', '4816.0', '46', '17']
['08/18', '025864', '141', '28718.0', '12941.0', '45', '4']
['08/18', '025606', '142', '29487.0', '-9629.0', '11', '6']
['08/18', '025606', '143', '26154.0', '3612.0', '18', '4']
['08/18', '025838', '144', '29231.0', '-20763.0', '5', '3']
['08/18', '025826', '145', '4359.0', '-2106.0', '1', '1']
['08/18', '025826', '146', '13333.0', '3913.0', '10', '2']
['08/18', '025764', '147', '21795.0', '-11735.0', '4', '2']
['08/18', '025764', '148', '21282.0', '-902.0', '11', '3']
['08/18', '025791', '149', '13333.0', '-5416.0', '3', '1']
['08/18', '025839', '150', '25128.0', '1506.0', '20', '5']
['08/18', '025557', '151', '0', '0', '0', '0']
['08/18', '025439', '152', '8462.0', '-5717.0', '1', '1']
['08/18', '025822', '153', '27436.0', '14746.0', '25', '5']
['08/18', '025567', '154', '16667.0', '-6018.0', '11', '4']
['08/18', '025593', '155', '9487.0', '-6619.0', '1', '1']
['08/18', '025827', '156', '20513.0', '14144.0', '23', '5']
['08

['08/18', '025847', '279', '10513.0', '-5717.0', '16', '4']
['08/18', '025847', '280', '15385.0', '-11434.0', '13', '10']
['08/18', '025700', '281', '10513.0', '-7522.0', '1', '1']
['08/18', '025700', '282', '3333.0', '-902.0', '5', '1']
['08/18', '025544', '283', '9744.0', '-1203.0', '24', '6']
['08/18', '025544', '284', '11795.0', '2709.0', '41', '4']
['08/18', '025703', '285', '1282.0', '4214.0', '10', '1']
['08/18', '025758', '286', '9744.0', '-6920.0', '2', '2']
['08/18', '025761', '287', '1795.0', '904.0', '8', '1']
['08/18', '025761', '288', '1538.0', '1506.0', '9', '1']
['08/18', '025879', '289', '5641.0', '5117.0', '11', '2']
['08/18', '025879', '290', '27179.0', '13844.0', '37', '6']
['08/18', '025737', '291', '6410.0', '-5115.0', '0', '0']
['08/18', '025737', '292', '10256.0', '-902.0', '17', '2']
['08/18', '025737', '293', '14103.0', '-6920.0', '9', '3']
['08/18', '025636', '294', '11538.0', '-2707.0', '7', '3']
['08/18', '025636', '295', '13333.0', '4214.0', '17', '2']
['0

['08/18', '025856', '421', '21538.0', '11737.0', '20', '2']
['08/18', '025856', '422', '25385.0', '12941.0', '23', '2']
['08/18', '025856', '423', '25641.0', '-12337.0', '6', '3']
['08/18', '025856', '424', '29487.0', '-18656.0', '4', '3']
['08/18', '025856', '425', '9231.0', '13242.0', '14', '1']
['08/18', '025856', '426', '23590.0', '-12337.0', '5', '3']
['08/18', '025856', '427', '15641.0', '-9027.0', '3', '3']
['08/18', '025856', '428', '16154.0', '9029.0', '15', '2']
['08/18', '025606', '429', '6923.0', '8427.0', '9', '1']
['08/18', '025606', '430', '6410.0', '-4513.0', '1', '1']
['08/18', '025606', '431', '6410.0', '7524.0', '8', '2']
['08/18', '025606', '432', '4359.0', '-3309.0', '1', '1']
['08/18', '025606', '433', '1795.0', '-1504.0', '0', '0']
['08/18', '025606', '434', '5897.0', '-902.0', '4', '2']
['08/18', '025606', '435', '7949.0', '-4814.0', '3', '3']
['08/18', '025606', '436', '16923.0', '-8124.0', '4', '2']
['08/18', '025606', '437', '4872.0', '-3610.0', '0', '0']
['0

['08/18', '120122', '557', '1588', '-1250.0', '6', '2', '--']
['08/18', '120122', '558', '277', '-231.0', '1', '1', '--']
['08/18', '120122', '559', '6881', '3565.0', '97', '32', '--']
['08/18', '120122', '560', '1187', '787.0', '22', '2', '--']
['08/18', '120073', '561', '4383', '-2546.0', '0', '17', '20']
['08/18', '120073', '562', '6418', '787.0', '0', '21', '48']
['08/18', '120073', '563', '5844', '463.0', '0', '15', '43']
['08/18', '120073', '564', '4207', '-3287.0', '0', '9', '17']
['08/18', '120073', '565', '3865', '-833.0', '0', '11', '22']
['08/18', '120073', '566', '7163', '-139.0', '0', '23', '43']
['08/18', '120073', '567', '5530', '-2824.0', '0', '16', '29']
['08/18', '120073', '568', '3946', '-972.0', '0', '10', '24']
['08/18', '120077', '569', '8545', '1713.0', '0', '36', '80']
['08/18', '120077', '570', '9939', '1111.0', '0', '33', '70']
['08/18', '120077', '571', '3654', '-926.0', '0', '10', '20']
['08/18', '120077', '572', '4340', '324.0', '0', '15', '38']
['08/18', '

['08/18', '120126', '692', '2309', '-1528.0', '0', '7', '6']
['08/18', '120125', '693', '2626', '-556.0', '8', '4', '--']
['08/18', '120125', '694', '7499', '1481.0', '34', '16', '--']
['08/18', '120125', '695', '7488', '741.0', '30', '20', '--']
['08/18', '120125', '696', '1768', '-926.0', '2', '5', '--']
['08/18', '120125', '697', '4534', '324.0', '17', '10', '--']
['08/18', '120125', '698', '2466', '-602.0', '7', '4', '--']
['08/18', '120125', '699', '1351', '-694.0', '2', '4', '--']
['08/18', '120125', '700', '8658', '93.0', '28', '26', '--']
['08/18', '120125', '701', '4696', '880.0', '19', '12', '--']
['08/18', '120125', '702', '3199', '-1250.0', '5', '9', '--']
['08/18', '120125', '703', '5213', '278.0', '18', '15', '--']
['08/18', '120125', '704', '565', '-509.0', '1', '0', '--']
['08/18', '120125', '705', '4404', '417.0', '15', '11', '--']
['08/18', '120025', '706', '4012', '926.0', '17', '9', '--']
['08/18', '120110', '707', '4274', '-1481.0', '19', '13', '--']
['08/18', '120

['08/18', '119897', '825', '5518', '-833.0', '40', '0', '27']
['08/18', '120070', '826', '5052', '3148.0', '0', '12', '73']
['08/18', '120088', '827', '4520', '-4398.0', '5', '13', '7']
['08/18', '119950', '828', '3974', '-1157.0', '0', '7', '32']
['08/18', '119824', '829', '4789', '-1065.0', '0', '0', '30']
['08/18', '119973', '830', '5998', '-556.0', '0', '14', '47']
['08/18', '120083', '831', '4419', '-1389.0', '0', '23', '23']
['08/18', '120075', '832', '6716', '-2639.0', '0', '18', '29']
['08/18', '119909', '833', '4603', '-3102.0', '8', '0', '3']
['08/18', '120109', '834', '5157', '-741.0', '0', '7', '47']
['08/18', '119987', '835', '3683', '-1713.0', '0', '0', '31']
['08/18', '119987', '836', '4962', '-556.0', '0', '0', '53']
['08/18', '120080', '837', '3086', '648.0', '22', '0', '34']
['08/18', '120127', '838', '7600', '-3194.0', '0', '1', '15']
['08/18', '120051', '839', '4775', '463.0', '0', '20', '37']
['08/18', '119864', '840', '2015', '-972.0', '7', '0', '2']
['08/18', '11

{'spreadsheetId': '1oEspZKFeuIqrdivdDBiI2HXvIshXP3NM_4tsDw2fThQ',
 'updates': {'spreadsheetId': '1oEspZKFeuIqrdivdDBiI2HXvIshXP3NM_4tsDw2fThQ',
  'updatedRange': 'S2.2RAW!F10294:M10341',
  'updatedRows': 48,
  'updatedColumns': 8,
  'updatedCells': 384}}